<a href="https://colab.research.google.com/github/FrancescoCortopassi/copilot-codespaces-vscode/blob/main/MT5_LEZIONE_23_to_32.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LEZIONE_23: GLI INDICATORI TECNICI

+I TI (Technical Indicators) sono dei calcoli matematici basati su movimento dei prezzi,
volumi o altre informazioni utiuli per ipotizzare il trend di uno strumento finanziario

+La maggior parte sono indicatori "lagging", ovvero in ritardo, e come detto sono utili per la previsione di un trend

+Ovviamente questi dati non prendono in considerazione l'analisi fondamentale,seguendo e cercando di sfruttare invece la teoria dei mercati efficienti (in cui sis ostiene che i prezzi degli asset contengano tutte le informazioni disponibili)

+Gli indicatori tecnici più popolari sono:
-MACD
-Bollinger Bands
-RSI
-ADX
-Supertrend
-Renko Charts


#LEZIONE_24: Indicatori tecnici in MT5
In questa lezione andiamo a vedere come reperire ed utilizzare gli indicatori su MT5, ed anche come ottenere indicatori che non sono caricati sul terminale di default

1.È importante sottolineare come tutti i codici degli indicatori che possiamo selezionare su MT5 siano scritti in linguaggio MQL5, quindi una cosa interessante da poter fare per implementarli in Python è andare a leggere il loro funzionamento tramite (tasto destro----view) per comprendere la logica dell'indicatore e ricrearla con Python

2.La raccomandazione del docente è comunque quella di costruire i propri indicatori direttamente con python senza dover tradurre codice MQL5

3.TRADINGVIEW è considerata la base di studio degli indicatori tecnici, e possiamo trovare indicazioni su come sviluppare tutti gli indicatori in quasi tutti i linguaggi di programmazione esistenti (ovviamente compreso Python)


LEZIONE_25: MOVING AVERAGE
Andiamo ad analizzare l'indicatore tecnico SMA (Media Mobile Semplice), che in realtà è una famiglia di indicatori.
Andiamo a vedere i due indicatori più utilizzati di questa famiglia:
1.SIMPLE MOVING AVERAGE: Un calcolo matematico che sviluppa medie dei prezzi passati secondi diversi range temporali, in cui ogni dato ha la stessa rilevanza per il calcolo della media. Generalmente per un sistema di trading si sviluppano 2 MEDIE MOBILI, quella che va più indietro nel tempo viene chiamata FAST MA, mentre quella che analizza un range di prezzi più recente si chiama FAST MA
2.EXPONENTIAL MOVING AVERAGE: Un calcolo matematico simile a quello della MA, ma che da più valore ai prezzi più recenti (per questo si chiama Media Mobile Esponenziale)

In [ ]:
#Funzione per calcolare l'SMA
ser=hist_data.close
def SMA (ser, n=9):
    return ser.rolling(30).mean()

#Funzione per calcolare l'EMA
def EMA (ser,n=9):
    multiplyer = 2/(n+1)
    sma =SMA (ser,n=9)
    ema= np.full (len(ser),np.nan)
    ema[len(sma)-len(sma.dropna())]=sma.dropna()[0]#questo codice permette al rprimo valore generato dalla SMA di essere anche il primo valore dell'EMA (questa cosa non l'ho compresa molto bene, DA APPROFONDIRE)
    for i in range (len(ser)):
        if not np.isnan(ema[i-1]):
            ema[i]=ema (ser.iloc[i]-ema[-1]*multiplyer+ ema[i-1])


hist_data["ema"] = EMA (hist_data.close)

In [ ]:
#LEZIONE_26: IMPLEMENTAZIONE MA
#la prima cosa che dobbiamo fare è eseguire la funzione "get_hist_data_numeric_index" che sarebbela funzione di recupero dei dati
#in modalità TIMEZONE AGNOSTIC (ovvero che non ha bisogno di specificare o regolare la TIMEZONE)
import pandas as pd
import MetaTrader5 as mt5
import pytz


def get_hist_data_numeric_index(symbol,timeframe,start_pos=0,num_candles=200):
  hist_data=mt5.copy_rates_from_pos(symbol,(datetime,timeframe),start_pos=0,num_candles=200)
  hist_data_dataframe=pd.DataFrame(hist_data)
  hist_data_df.time=pd_to_datetime(hist_data_df.time,unit="s")
  hist_data_df.set_index ["time",inplace==True]
  return hist_data_df

hist_data=get_hist_data_numeric_index("USDCAD","H1",num_candles=100)

#LEZIONE 27: MACD

Il MACD è un indicatore di TREND FOLLOWING MOMENTUM ed è ampiamente utilizzato per confermare una variazione di trend. Generalmente è composta da due MEDIE MOBILI ESPONENZIALI (generalmente a 12 ed a 26 periodi))


Una signal line (di solito a 9 periodi) viene utilizzata per generare i segnali


Quando il MACD taglia la signal line dal basso verso l'alto si tratta di un segnale BUY, viceverrsa si tratta di un segnale SELLL

Da molti falsi positivi, da non utilizzare come unico segnale di trading

Segnale soggetto a LAGGING (RITARDO)

SI SPIEGA POI COME IMPLEMENTARLO SU MT5






In [ ]:
#LEZIONE 28: MACD IMPLEMENTATION
#andiamo a crare la funzione per il MACD che poi potremo riutilizzare
#i parametri da passare nella funzione sono il DATAFRAME, i due periofdi per le
#MEDIE MOBILI ESPONENZIALI (12,26)ed il periodo sulla SIGNAL LINE (9)

def MACD (df, a=12, b=26, c=9):
    df=DF.copy #creiamo una copia del DataFrame
    df [ma_fast]=EMA (df["close"],a)
    df [ma_slow]=EMA (df["close"],b)
    df [Macd]= df ["ma_fast"] - df ["ma_slow"]
    df [signal]= EMA (df["macd"],c)
    df [histogram] =df ["macd"]- df ["signal"]
    return df["macd","signal","histogram"]

hist_data[["macd","signal","histogram"]]=MACD (hist_data)


LEZIONE_29: RSI

L'RSI è un'OSCILLATORE DI MOMENTUM che calcola la velocità e la forza con cui un prezzo si muove all'interno di un RANGE predefinito

Il valore dell'RSI all'interno del range oscilla tra 0 e 100, con delle sogli a 70 ed a 30 che indicano rispettizone di IPERCOMPRATO ed IPERVENDUTO. In alcuni mercati emergenti si usa la proporzione 80/20 (le proporzioni possono essere testate a discrezionalità del trader)

È importante ricordarsi che gli asset possono rimanere in ipercomprato o in ipervenduto per lunghi periodi di tempo, quindi questo indicatore non dovrebbe essere utilizzato da solo ma come conferma di altri segnali (ex: inversione di un trend)

Calcolare la qualità di un'ingresso a mercato utilizzando solo l'RSI può risultare complicato e non profittevole

Gli steps descritti su Tradingview per il calcolo sono i seguenti:
1. Calcolare i cambiamenti di prezzo in un range prestabilito alla sua chiusura (close)
2. Dobbiamo calcolare GAIN e LOSS, quindi se ad ogni chiusura dell'asset il prezzo si è apprezzato o deprezzato rispetto al giorno precedente
3. Il calcolo dell' RS (la forza relativa dell'asset) prevede un rapporto (divisione) tra perdite e vincite nel periodo di tempo preso in considerazione (in questo caso si tratterebbe di 14 giorni presi in considerazione)
4. Infine il calcolo dell'RSI si ottiene con questa formula (100-(100:1+RS))


In [ ]:
#Funzione per calcolare l'RMA, ripresa dalla funzione precedente per calcolare
def RMA (ser,n=9):
    multiplyer = 1/n
    sma =ser.rolling (n).mean()
    ema= np.full (len(ser),np.nan)
    ema[len(sma)-len(sma.dropna())]=sma.dropna()[0]#questo codice permette al rprimo valore generato dalla SMA di essere anche il primo valore dell'EMA (questa cosa non l'ho compresa molto bene, DA APPROFONDIRE)
    for i in range (len(ser)):
        if not np.isnan(ema[i-1]):
            ema[i]=ema (ser.iloc[i]-ema[-1]*multiplyer+ ema[i-1])

#LEZIONE_29: RSI IMPLEMENTATION
def RSI (DF, n=5):
  #inseriamo come valori di funzione il DaatFrame ed il numero di periodi da
  #prendere in considerazione: noi abbiamo inserito il valore che MT5 ci da di default, ovvero 5
  df= DF.copy()
  change = df["close"]-df ["close"].shift (1)
  df["gain"]= np.where (df ["change"]>0, df ["change"],0)#si usa numpy.where definendo quali sono i valori per cui la condizione è rispettata e quelli per cui non lo è
  df["loss"]= np.where (df["change"]<0, -1* df ["change"],0)#non ho capito perché è stato inserito questo valore di -1, DA RIVEDERE
  df ["avg_gain"]= (df ["gain"],n)
  df ["avg_loss"]= (df ["loss"],n)
  df ["rs"]= (df[avg_gain]/df[avg_loss])
  df ["rsi"]=100-(100/(1+df["rs"]))#come vediamo questa è la formula per il calcolo dell' RSI che abbiamo visto all'inizio
  return df ["rsi"]

hist_data["rsi"]=RSI (hist_data)





LEZIONE_30: INDICATORE STOCASTICO (STOCHASTIC)

L'indicatore STOOCHASTIC è un'oscillatore utilizzato per calcolare la velocità del momentum dei prezzi

La formula per calcolarlo è la seguente: (Close-Lowest Low)/(Highest high-Lowest Low) * 100

I valori variano da 0 a 100, considerando che un valore massimo vicino al 100 indicherebbe che il prezzo ha raggiunto il suo massimo valore di incremento nella finestra temporale presa in considerazione

Sopra gli 80 indica ipercomprato e sotto i 20 indica ipervenduto

Calcolare il momentum diventa importante sulla base dell'asseverazione che prima di un'inversione di tendeza (reversal) il momentum deve cambiare. Questo è un ragionamento che funziona meglio quando siamo in un trend market

In [ ]:
def Stochastic (DF, loockbac=14, k=3, d=3)
df= DF.copy()
df ["HH"]=df ["close"].rollig (loockback).max()
df ["LL"]=df ["close"].rollig (loockback).min()
df ["%K"]=((df ["close"]-df ["LL"])/(df["HH"]-df ["LL"])).rollig (k).mean()
df ["%D"]=df ["%K"].rollig (d).mean()
return df ["%K","%D"]

hist_data ["%K","%D"]=Stochastic (hist_data)

#((df ["close"]-df ["LL"])/(df["HH"]-df ["LL"])).rollig (k).mean()///////#questa è la formula per il calcolo dell'inidcatore Stocastico trasformata codice Python

LEZIONE_31: SUPPORTI E RESISTENZE (parte 1)

 I supporti e resistenze sono LIVELLI IMMAGINARI in cui il prezzo verifica una reale difficoltà a passare.

 Ci sono diversi approcci che i trader quantitativi utilizzano per definire supporti e resistenze

 La funzione che andremo a sviluppare per tracciare supporti e resistenze è basata sul concetto di "Hills & Valleys" (colline e valli)



In [ ]:
from os import remove
from ast import Index
LEZIONE_31: SUPPORTI E RESISTENZE (parte 1)
#andiamo a creare la funzione per definire "hilss&valleys"

def get_hills_valleys (DF, pip=0.0001):
  df=DF.copy()
  df ["hill_valley"]= np.where(np.logical_and(df.low < df.low(shift=(1) - 3*pip), #la funzione NP.WHERE viene utilizzata per selezionare elementi di un'array in base a una condizione
                                                df.low < df.low(shift=(2) - 3*pip )
                                               ),
                        np.logical_and((df.low < df.low(shift=(-1) - 3*pip), #la funzione LOGICAL_AND si utilizza per applicare un AND LOGICO ad un processo che coinvolge la combinazione di più condizioni su un'array Python
                                        df.low < df.low(shift=(-2) - 3*pip )
                                       )),"valley") #questo codice permette di riconoscere nel nostro DF tutti i punti (del range selezionato) considerati dalla nostra funzione delle "valleys"
                        np.where(np.logical_and(df.high > df.high(shift=(1) + 3*pip),
                                                df.high > df.high(shift=(2) + 3*pip )
                                               ),
                                (np.logical_and(df.high > df.high(shift=(1) + 3*pip),
                                                df.high > df.high(shift=(2) + 3*pip )
                                               ),"hill") #stessa cosa per creare
                                 return df ["hill_valley"]


def get_support (df, list_of_lows, current_price, level_range=20, pip=0.0001):
  support_levels= []
  remove_index= [] #immagazzina tutti gli indici che sono stati usati per definire un valido supporto
  for indx, low in enumerate (list_of_lows):
    if indx in remove_index:
        continue
    count=0
    lows = []
    for j in list_of_lows [indx]:
        if abs (low-j)< level_range*pip:
          count+=1
          lows.append(j)
        if count> min_occurance:
           suppor_levels.append(sum(lows)/len(lows)) if sum(lows)/len(lows) < current_price else support_levels
           #adesso dobbiamorimuovere tutti gli indici di un valore usati per calcolare il livello di supporto identificato
           for k in lows:
               remove index (list_of_lows(k)) if list_of_lows.index (k) not in remove_index else remove
        if len(support_levels)==0
          return min (list_of_lows)
          else:
               return current_price

          elif list_of_lows.index (min(list_of_lows)) > 0.7 (len(list_of_lows)):
            return min(list_of_lows)
          else:
            return sorted (support_levels, reverse= True) [0]







